In [ ]:
#%reload_ext autoreload
#%autoreload 2

In [ ]:
from nbdev import *
%nbdev_default_export nodes

Cells will be exported to pct.nodes,
unless a different module is specified after an export flag: `%nbdev_export special.module`


# Nodes


In [ ]:
# hide
import sys
sys.path.append("..")

In [ ]:
from pct.functions import *

In [ ]:
def velocity_model(velocity,  force , mass):
    velocity = velocity + force / mass
    return velocity

In [ ]:
%nbdev_export
class PCTNodeData():
    "Data collected for a PCTNode"
    def __init__(self, name="pctnodedata"):
        self.data = {
            "refcoll":{}, 
            "percoll":{},
            "comcoll":{}, 
            "outcoll":{}}
        
    
    def add_data(self, node):
        ctr = 0 
        
        self.add_collection( node.referenceCollection, "refcoll")
        self.add_collection( node.perceptionCollection, "percoll")
        self.add_collection( node.comparatorCollection, "comcoll")
        self.add_collection( node.outputCollection, "outcoll")

    def add_collection(self, collection, collname):
        for func in collection:            
            if self.data[collname].get(func.get_name()) == None:
                dlist=[]
                cdict={func.get_name():dlist}
                self.data[collname]=cdict
            else:
                dlist = self.data[collname][func.get_name()]
                
            dlist.append(func.get_value())

In [ ]:
%nbdev_export
class PCTNode():
    "A single PCT controller."
    def __init__(self, perception, name="pctnode", history=False):
        self.links_built = False
        if history:
            self.history = PCTNodeData()
        self.name=name 
        self.perceptionCollection = [perception]
        reference = Constant(1)
        self.referenceCollection = [reference]
        comparator = Subtract()
        self.comparatorCollection = [comparator]
        self.outputCollection = [Proportional(10)]
    
    def __call__(self, verbose=False):
        if not self.links_built:
            self.build_links()
            
        for referenceFunction in self.referenceCollection:
            referenceFunction(verbose)               

        for perceptionFunction in self.perceptionCollection:
            perceptionFunction(verbose)
                    
        for comparatorFunction in self.comparatorCollection:
            comparatorFunction(verbose)

        for outputFunction in self.outputCollection:
            outputFunction(verbose)
            
        self.output = self.outputCollection[-1].get_value()
        
        if verbose:
            print()
            
        if not self.history == None:
            self.history.add_data(self)
            
        return self.output
    
    def build_links(self):
        if len(self.referenceCollection)>0:
            link = self.referenceCollection[0]
            for i in range (1, len(self.referenceCollection)):
                self.referenceCollection[i].add_link(link)               
                link = self.referenceCollection[i]

        if len(self.perceptionCollection)>0:
            link = self.perceptionCollection[0]
            for i in range (1, len(self.perceptionCollection)):
                self.perceptionCollection[i].add_link(link)               
                link = self.perceptionCollection[i]

        self.comparatorCollection[0].add_link(self.referenceCollection[-1])
        self.comparatorCollection[0].add_link(self.perceptionCollection[-1])

        if len(self.comparatorCollection)>1:
            link = self.comparatorCollection[1]
            for i in range (1, len(self.comparatorCollection)):
                self.comparatorCollection[i].add_link(link)               
                link = self.comparatorCollection[i]

        self.outputCollection[0].add_link(self.comparatorCollection[-1])

        if len(self.outputCollection)>0:
            link = self.outputCollection[0]
            for i in range (1, len(self.outputCollection)):
                self.outputCollection[i].add_link(link)               
                link = self.outputCollection[i]

        self.links_built = True

    def run(self, steps=None, verbose=False):
        for i in range(steps):
            out = self(verbose)
        return out
    
    def set_output(self, value):
        self.outputCollection[-1].set_value(value)
        
    def get_output_function(self):
        return self.outputCollection[-1]
    
    
    def set_function_name(self, collection, position, name):
        if collection == "refcoll":
            self.referenceCollection[position].set_name(name)
    
    def summary(self):
        if not self.links_built:
            self.build_links()

        print(self.name, type(self).__name__)
        print("----------------------------")
        print("REF:", end=" ")
        for referenceFunction in self.referenceCollection:
            referenceFunction.summary()   
        
        print("PER:", end=" ")
        for perceptionFunction in self.perceptionCollection:
            perceptionFunction.summary()
        
        print("COM:", end=" ")
        for comparatorFunction in self.comparatorCollection:
            comparatorFunction.summary()
        
        print("OUT:", end=" ")
        for outputFunction in self.outputCollection:
            outputFunction.summary()
        
        print("----------------------------")


In [ ]:
mass = 50
force = 0
vel = Variable(0, name="velocity")
node = PCTNode(vel, history=True)

In [ ]:
node.summary()

pctnode PCTNode
----------------------------
REF: constant Constant | 1 
PER: velocity Variable | 0 
COM: subtract Subtract | 0 | links  constant velocity 
OUT: proportional Proportional | gain 10 | 0 | links  subtract 
----------------------------


In [ ]:
output = node(verbose=False)
force = output
vel.set_value(velocity_model(vel.get_value(), force, mass))
print(force)
assert output == 10

10


In [ ]:
velocity_input = Variable(0, name="velocity")
pctnode = PCTNode(velocity_input, history=True)
pctnode.set_function_name("refcoll", -1, "reference")

for i in range(40):
    force = pctnode(verbose=False)
    vel = velocity_model(velocity_input.get_value(), force, mass)
    velocity_input.set_value(vel)

In [ ]:
import plotly.graph_objects as go
fig = go.Figure(layout_title_text="Velocity Goal")
fig.add_trace(go.Scatter(y=pctnode.history.data['refcoll']['reference'], name="ref"))
fig.add_trace(go.Scatter(y=pctnode.history.data['percoll']['velocity'], name="perc"))

In [ ]:
#hide
from nbdev import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_rmath.ipynb.
Converted 02_functions.ipynb.
Converted 03_nodes.ipynb.
Converted 04_hierarchy.ipynb.
Converted index.ipynb.
Converted Untitled.ipynb.
